In [1]:
import numpy as np

In [2]:
from tqdm import tqdm

In [3]:
import pandas

In [4]:
import pyiron_contrib

In [5]:
from pyiron_base import state

In [6]:
from pyiron_atomistics import Project

In [7]:
def body_order(n=32, b=5):
    if b == 2:
        return [[i ,n-i] for i in range(n+1)]
    else:
        lst = []
        for i in range(n+1):
            for j in body_order(n=n-i, b=b-1):
                lst.append([i] + j)
        return lst

In [8]:
def get_elastic_constants(conc_lst, number_atoms, element_lst, potential, structure):
    # create project 
    project = Project('workflow')
    
    # Generate structure 
    if sum(np.array(conc_lst)==0) != 4:
        mole_fraction = {
            el: conc/number_atoms
            for el, conc in zip(element_lst, conc_lst)
            if conc > 0
        }
        job = project.create_job(project.job_type.SQSJobWithoutOutput, "sqs")
        job._interactive_disable_log_file = True
        job.structure = structure
        job.input['mole_fractions'] = mole_fraction
        job.input['iterations'] = 1e6
        job.server.cores = 1
        job.run()
        structure_next = job._lst_of_struct[-1]
    else:
        structure_next = structure.copy()
        structure_next.symbols[:] = np.array(element_lst)[np.array(conc_lst)!=0][0] 
    
    # Minimize strucute 
    lmp_mini1 = project.create_job(project.job_type.LammpsInteractiveWithoutOutput, "lmp_mini", delete_existing_job=True)
    lmp_mini1.structure = structure_next
    lmp_mini1.potential = potential
    lmp_mini1.calc_minimize(pressure=0.0)
    lmp_mini1.server.run_mode.interactive = True
    lmp_mini1._interactive_disable_log_file = True  # disable lammps.log 
    lmp_mini1.run()
    lmp_mini1.interactive_close()

    # Elastic constants 
    lmp_elastic = project.create_job(project.job_type.LammpsInteractiveWithoutOutput, "lmp_elastic", delete_existing_job=True)
    lmp_elastic.structure = lmp_mini1.get_structure()
    lmp_elastic.potential = potential
    lmp_elastic.interactive_enforce_structure_reset = True 
    lmp_elastic.server.run_mode.interactive = True
    lmp_elastic._interactive_disable_log_file = True  # disable lammps.log
    elastic = lmp_elastic.create_job(project.job_type.ElasticMatrixJobWithoutFiles, "elastic", delete_existing_job=True)
    elastic._interactive_disable_log_file = True  # disable lammps.log
    elastic.run()
    
    elastic_constants_lst = [
        elastic._data["C"][0][0],
        elastic._data["C"][0][1],
        elastic._data["C"][0][2],
        elastic._data["C"][0][3],
        elastic._data["C"][0][4],
        elastic._data["C"][0][5],
        elastic._data["C"][1][1],
        elastic._data["C"][1][2],
        elastic._data["C"][1][3],
        elastic._data["C"][1][4],
        elastic._data["C"][1][5],
        elastic._data["C"][2][2],
        elastic._data["C"][2][3],
        elastic._data["C"][2][4],
        elastic._data["C"][2][5],
        elastic._data["C"][3][3],
        elastic._data["C"][3][4],
        elastic._data["C"][3][5],
        elastic._data["C"][4][4],
        elastic._data["C"][4][5],
        elastic._data["C"][5][5],
    ]

    if "Fe" in elastic.ref_job.structure.get_species_symbols():
        conc_Fe = sum(elastic.ref_job.structure.indices==elastic.ref_job.structure.get_species_symbols().tolist().index("Fe")) / len(elastic.ref_job.structure.indices)
    else:
        conc_Fe = 0.0
    if "Ni" in elastic.ref_job.structure.get_species_symbols():
        conc_Ni = sum(elastic.ref_job.structure.indices==elastic.ref_job.structure.get_species_symbols().tolist().index("Ni")) / len(elastic.ref_job.structure.indices)
    else:
        conc_Ni = 0.0
    if "Cr" in elastic.ref_job.structure.get_species_symbols():
        conc_Cr = sum(elastic.ref_job.structure.indices==elastic.ref_job.structure.get_species_symbols().tolist().index("Cr")) / len(elastic.ref_job.structure.indices)
    else:
        conc_Cr = 0.0
    if "Co" in elastic.ref_job.structure.get_species_symbols():
        conc_Co = sum(elastic.ref_job.structure.indices==elastic.ref_job.structure.get_species_symbols().tolist().index("Co")) / len(elastic.ref_job.structure.indices)
    else:
        conc_Co = 0.0
    if "Cu" in elastic.ref_job.structure.get_species_symbols():
        conc_Cu = sum(elastic.ref_job.structure.indices==elastic.ref_job.structure.get_species_symbols().tolist().index("Cu")) / len(elastic.ref_job.structure.indices)
    else:
        conc_Cu = 0.0
        
    conc_lst = [conc_Fe, conc_Ni, conc_Cr, conc_Co, conc_Cu]
    
    return elastic_constants_lst + conc_lst

In [9]:
state.settings.configuration

{'user': 'pyiron',
 'resource_paths': ['/Users/janssen/pyiron/resources',
  '/Users/janssen/mambaforge/share/pyiron'],
 'project_paths': [],
 'connection_timeout': 60,
 'sql_connection_string': None,
 'sql_table_name': 'jobs_pyiron',
 'sql_view_connection_string': None,
 'sql_view_table_name': None,
 'sql_view_user': None,
 'sql_view_user_key': None,
 'sql_file': '/Users/janssen/pyiron.db',
 'sql_host': None,
 'sql_type': 'SQLite',
 'sql_user_key': None,
 'sql_database': None,
 'project_check_enabled': False,
 'disable_database': True,
 'credentials_file': None,
 'write_work_dir_warnings': True,
 'config_file_permissions_warning': True}

In [10]:
structure = Project('.').create.structure.ase.bulk("Al", cubic=True).repeat([2,2,2])

In [11]:
element_lst = ["Fe", "Ni", "Cr", "Co", "Cu"]

In [12]:
number_atoms = len(structure)

In [13]:
number_species = len(element_lst)

In [14]:
potential = '2021--Deluigi-O-R--Fe-Ni-Cr-Co-Cu--LAMMPS--ipr1'

In [15]:
lst_32 = body_order(n=number_atoms, b=number_species)[:10]

In [16]:
c11_lst, c12_lst, c13_lst, c14_lst, c15_lst, c16_lst, c22_lst, c23_lst, c24_lst, c25_lst, c26_lst, c33_lst, c34_lst, c35_lst, c36_lst, c44_lst, c45_lst, c46_lst, c55_lst, c56_lst, c66_lst = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []  

In [17]:
conc_Fe_lst, conc_Ni_lst, conc_Cr_lst, conc_Co_lst, conc_Cu_lst = [], [], [], [], []

In [18]:
for conc_lst in tqdm(lst_32):
    c11, c12, c13, c14, c15, c16, c22, c23, c24, c25, c26, c33, c34, c35, c36, c44, c45, c46, c55, c56, c66, conc_Fe, conc_Ni, conc_Cr, conc_Co, conc_Cu = get_elastic_constants(
        conc_lst=conc_lst, 
        number_atoms=number_atoms, 
        element_lst=element_lst, 
        potential=potential, 
        structure=structure
    )
    
    c11_lst.append(c11)
    c12_lst.append(c12)
    c13_lst.append(c13) 
    c14_lst.append(c14)
    c15_lst.append(c15)
    c16_lst.append(c16)
    c22_lst.append(c22) 
    c23_lst.append(c23) 
    c24_lst.append(c24) 
    c25_lst.append(c25) 
    c26_lst.append(c26) 
    c33_lst.append(c33) 
    c34_lst.append(c34)
    c35_lst.append(c35)
    c36_lst.append(c36)
    c44_lst.append(c44)
    c45_lst.append(c45)
    c46_lst.append(c46)
    c55_lst.append(c55)
    c56_lst.append(c56)
    c66_lst.append(c66)
    
    conc_Fe_lst.append(conc_Fe)
    conc_Ni_lst.append(conc_Ni)
    conc_Cr_lst.append(conc_Cr)
    conc_Co_lst.append(conc_Co)
    conc_Cu_lst.append(conc_Cu)

  0%|                                                    | 0/10 [00:00<?, ?it/s]

The job lmp_mini was saved and received the ID: lmp_mini


/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:121: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


The job elastic was saved and received the ID: elastic
The job elastic_lmp_elastic was saved and received the ID: elastic_lmp_elastic


/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
 10%|████▍                                       | 1/10 [00:02<00:19,  2.22s/it]

...

/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
 90%|███████████████████████████████████████▌    | 9/10 [00:33<00:04,  4.37s/it]

The job sqs was saved and received the ID: sqs
The job lmp_mini was saved and received the ID: lmp_mini


/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:121: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


The job elastic was saved and received the ID: elastic
The job elastic_lmp_elastic was saved and received the ID: elastic_lmp_elastic


/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
100%|███████████████████████████████████████████| 10/10 [00:37<00:00,  3.78s/it]


In [19]:
df = pandas.DataFrame({
    "conc_Fe": conc_Fe_lst, 
    "conc_Ni": conc_Ni_lst, 
    "conc_Cr": conc_Cr_lst, 
    "conc_Co": conc_Co_lst, 
    "conc_Cu": conc_Cu_lst,
    "C11": c11_lst, 
    "C12": c12_lst, 
    "C13": c13_lst,
    "C14": c14_lst, 
    "C15": c15_lst, 
    "C16": c16_lst, 
    "C22": c22_lst, 
    "C23": c23_lst, 
    "C24": c24_lst, 
    "C25": c25_lst, 
    "C26": c26_lst, 
    "C33": c33_lst, 
    "C34": c34_lst, 
    "C35": c35_lst, 
    "C36": c36_lst, 
    "C44": c44_lst, 
    "C45": c45_lst, 
    "C46": c46_lst, 
    "C55": c55_lst, 
    "C56": c56_lst, 
    "C66": c66_lst
})

In [20]:
df

,conc_Fe,conc_Ni,conc_Cr,conc_Co,conc_Cu,C11,C12,C13,C14,C15,...,C33,C34,C35,C36,C44,C45,C46,C55,C56,C66
0,0.0,0.0,0.0,0.00000,1.00000,171.550807,116.387656,116.387656,0.000000,0.000000,...,171.550807,0.000000,0.000000,0.000000,76.350076,0.000000,0.000000,76.350076,0.000000,76.350076
1,0.0,0.0,0.0,0.96875,0.03125,168.206377,117.342596,117.342596,0.000000,0.000000,...,168.206377,0.000000,0.000000,0.000000,78.135186,0.000000,0.000000,78.135186,0.000000,78.135186
2,0.0,0.0,0.0,0.93750,0.06250,166.234934,118.955277,118.955277,0.000000,0.000000,...,170.331774,0.000000,0.000000,0.000000,79.999227,0.000000,0.000000,79.969331,0.000000,79.969330
3,0.0,0.0,0.0,0.90625,0.09375,169.830494,120.111975,120.096717,0.000000,0.000000,...,172.345130,0.000000,0.000000,0.143809,81.911329,0.020541,0.000000,81.911328,0.000000,81.880256
4,0.0,0.0,0.0,0.87500,0.12500,174.056252,122.159376,121.950598,-0.041975,0.063666,...,175.446023,0.066813,0.069999,0.031596,83.997544,-0.026272,0.062038,83.784203,-0.003053,83.893679
5,0.0,0.0,0.0,0.84375,0.15625,173.195113,119.169463,119.283620,0.000000,0.000000,...,173.498799,0.000000,0.000000,0.196381,86.033710,-0.000852,0.000000,85.816518,0.000000,85.782744
6,0.0,0.0,0.0,0.81250,0.18750,173.773891,118.746816,118.670301,-0.116886,0.143338,...,176.919928,0.018437,0.141167,0.097407,87.694192,0.033029,-0.013658,87.852351,0.034397,87.941296
7,0.0,0.0,0.0,0.21875,0.78125,179.926530,121.664557,120.893544,-0.027509,0.080308,...,180.484478,0.016485,0.102654,0.064639,90.210761,0.038960,-0.121167,89.268558,-0.012724,90.131909
8,0.0,0.0,0.0,0.75000,0.25000,185.043314,122.506874,122.279133,-0.043876,-0.149721,...,184.883812,-0.003535,-0.158749,0.039594,91.908759,0.041296,-0.019674,92.082864,-0.037516,92.149537
9,0.0,0.0,0.0,0.71875,0.28125,182.848441,118.166108,117.771095,-0.107932,0.013757,...,181.534600,-0.259874,0.015717,-0.056969,94.268240,-0.031112,0.049143,94.224925,0.088150,94.406855
